In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaLLM
from langchain.schema import HumanMessage
import json
from collections import defaultdict
import re

In [2]:
# === Load and split a single paper ===
def load_and_split_pdf(file_path):
    """Load a PDF and split it into manageable chunks."""
    try:
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_documents(docs)
        return chunks
    except Exception as e:
        print(f"❌ Error loading PDF {file_path}: {e}")
        return []

In [3]:
# === Initialize LLM ===
def get_llm():
    """Initialize the Ollama LLM with the desired model."""
    try:
        return OllamaLLM(model="llama3.2:latest")
    except Exception as e:
        print(f"❌ Error initializing LLM: {e}")
        raise

In [4]:
def extract_parameters(llm, chunk, title):
    """Extract research parameters from a single chunk of text."""
    prompt = f"""You are a research assistant analyzing academic papers. Based on the title "{title}", 
extract key research parameters from the following text chunk. 

For each parameter, only extract information if it's explicitly mentioned in the text chunk.
If a parameter is not found, use null for its value.

Parameters to extract:
- dataset: What datasets were used in the research
- model: What models or algorithms were used
- evaluation_metrics: Metrics used to evaluate performance
- problem_statement: The research problem being addressed
- method: The approach or methodology used
- results: Key findings or outcomes of the research

Text chunk:
{chunk.page_content}

Return ONLY a valid JSON object with these keys, structured as:
{{
  "dataset": string or null,
  "model": string or null, 
  "evaluation_metrics": string or null,
  "problem_statement": string or null,
  "method": string or null,
  "results": string or null
}}
"""
    try:
        # OllamaLLM returns a string directly, not an object with content attribute
        response_text = llm.invoke([HumanMessage(content=prompt)])

        # Extract JSON from response (in case LLM includes extra text)
        json_match = re.search(r'({[\s\S]*})', response_text)
        if json_match:
            json_str = json_match.group(1)
            return json.loads(json_str)
        else:
            print("⚠️ No JSON found in response")
            return {}
    except Exception as e:
        print(f"⚠️ Error during parameter extraction: {e}")
        print(f"Raw response: {response_text if 'response_text' in locals() else 'No response'}")
        return {}

In [5]:
# === Process all chunks of a paper ===
def process_chunks(chunks, title):
    """Process all chunks of a paper and merge the extracted parameters."""
    all_params = []
    llm = get_llm()
    
    for i, chunk in enumerate(chunks):
        print(f"🔍 Processing chunk {i+1}/{len(chunks)}")
        params = extract_parameters(llm, chunk, title)
        if params:
            all_params.append(params)
    
    # Merge parameters from all chunks
    return merge_parameters(all_params)

def merge_parameters(params_list):
    """Merge parameters extracted from different chunks into a cohesive set."""
    if not params_list:
        return {}
    
    merged = defaultdict(list)
    param_keys = ["dataset", "model", "evaluation_metrics", "problem_statement", "method", "results"]
    
    # Collect all non-null values for each parameter
    for params in params_list:
        for key in param_keys:
            if params.get(key) and params[key] != "null":
                merged[key].append(params[key])
    
    # Combine values for each parameter, removing duplicates
    final_params = {}
    for key in param_keys:
        if not merged[key]:
            final_params[key] = None
        else:
            # Convert each item to string before adding to set to ensure hashability
            unique_values = list(set(str(item) for item in merged[key]))
            final_params[key] = "\n".join(unique_values)
    
    return final_params

In [6]:
# === Main function to process all papers ===
def process_papers(papers):
    """Process a list of papers, extracting parameters from each."""
    research_params = []
    
    for title, file_path in papers:
        print(f"\n📄 Processing paper: {title}")
        chunks = load_and_split_pdf(file_path)
        if not chunks:
            print(f"⚠️ No chunks extracted from {title}")
            continue
            
        params = process_chunks(chunks, title)
        research_params.append({
            "title": title,
            "parameters": params
        })
        
        # Print a summary of the extracted parameters
        print(f"\n✅ Extracted parameters for: {title}")
        for param, value in params.items():
            if value:
                print(f"  - {param}: {value[:100]}..." if len(str(value)) > 100 else f"  - {param}: {value}")
    
    return research_params

In [7]:
# === Save results to file ===
def save_results(research_params, output_file="research_parameters_new.json"):
    """Save the extracted parameters to a JSON file."""
    try:
        with open(output_file, 'w') as f:
            json.dump(research_params, f, indent=2)
        print(f"\n💾 Results saved to {output_file}")
    except Exception as e:
        print(f"❌ Error saving results: {e}")

In [8]:
if __name__ == "__main__":
    papers = [
        ("Automatic Detection and Classification of Cognitive Distortions in Mental Health Text", r"C:\Users\abhay\OneDrive\Desktop\research_\papers\shickel2020.pdf"),
        ("The Acceptability of Therapist-Assisted, Internet-Delivered Treatment for College Students", r"C:\Users\abhay\OneDrive\Desktop\research_\papers\travers2014.pdf"),
        ("The Efficacy of Cognitive Behavioral Therapy: A Review of Meta-analyses", r"C:\Users\abhay\OneDrive\Desktop\research_\papers\hofmann2012.pdf"),
        ("Cognitive behavioural therapy targeting cardiac anxiety post-myocardial infarction: results from two sequential pilot studies",r"C:\Users\abhay\OneDrive\Desktop\research_\papers\oeaf020.pdf"),
        ("Therapist-Assisted, Online (TAO) Intervention for Anxiety in CollegeStudents: TAO Outperformed Treatment as Usual", r"C:\Users\abhay\OneDrive\Desktop\research_\papers\benton2016.pdf"),
        ("The Effectiveness of Online Cognitive Behavioral Treatment in Routine Clinical Practice", r"C:\Users\abhay\OneDrive\Desktop\research_\papers\ruwaard2012.pdf"),
        ("Educational attainment and cognitive behavioral therapy treatment outcome in late-life depression: A secondary analysis of a randomized controlled trial", r"C:\Users\abhay\OneDrive\Desktop\research_\papers\1-s2.0-S0165032725005397-main.pdf"),
        ("UNIPDES - An internet-based transdiagnostic intervention for college students’ psychological symptoms: Evaluation of its development, usability and effectiveness: Study protocol",r"C:\Users\abhay\OneDrive\Desktop\research_\papers\1-s2.0-S2451865425000171-main.pdf"),
        ("Internet-based digital intervention to support the self-management of hypertension compared to usual care: results of the HALCYON randomized controlled trial",r"C:\Users\abhay\OneDrive\Desktop\research_\papers\s12872-025-04698-5.pdf"),
        ("Predicting the treatment outcomes of major depressive disorder interventions with baseline resting-state functional connectivity: a meta-analysis",r"C:\Users\abhay\OneDrive\Desktop\research_\papers\s12888-025-06728-0.pdf"),
        ("Adaptations of an online cognitive-behavioral therapy intervention for binge type eating disorders in publicly-insured and uninsured adults: a pilot study",r"C:\Users\abhay\OneDrive\Desktop\research_\papers\s12889-025-22494-w.pdf")
        # Add more papers here...
    ]
    
    research_params = process_papers(papers)
    save_results(research_params)


📄 Processing paper: Automatic Detection and Classification of Cognitive Distortions in Mental Health Text
🔍 Processing chunk 1/43
🔍 Processing chunk 2/43
🔍 Processing chunk 3/43
🔍 Processing chunk 4/43
🔍 Processing chunk 5/43
🔍 Processing chunk 6/43
🔍 Processing chunk 7/43
🔍 Processing chunk 8/43
🔍 Processing chunk 9/43
🔍 Processing chunk 10/43
🔍 Processing chunk 11/43
🔍 Processing chunk 12/43
🔍 Processing chunk 13/43
🔍 Processing chunk 14/43
🔍 Processing chunk 15/43
🔍 Processing chunk 16/43
🔍 Processing chunk 17/43
🔍 Processing chunk 18/43
🔍 Processing chunk 19/43
🔍 Processing chunk 20/43
🔍 Processing chunk 21/43
🔍 Processing chunk 22/43
🔍 Processing chunk 23/43
🔍 Processing chunk 24/43
🔍 Processing chunk 25/43
🔍 Processing chunk 26/43
🔍 Processing chunk 27/43
🔍 Processing chunk 28/43
🔍 Processing chunk 29/43
🔍 Processing chunk 30/43
🔍 Processing chunk 31/43
🔍 Processing chunk 32/43
🔍 Processing chunk 33/43
🔍 Processing chunk 34/43
🔍 Processing chunk 35/43
🔍 Processing chunk 36/43
🔍 